<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [2]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [3]:
# Write your code here
# Summary of data types and counts (non-null values)
print(df.info())

# Calculate missing values count per column
missing_values = df.isnull().sum()

# Display columns with missing values (optional: only those with any missing data)
missing_values = missing_values[missing_values > 0]

print("\nMissing values per column:")
print(missing_values)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB
None

Missing values per column:
RemoteWork             10631
CodingActivities       10971
EdLevel                 4653
LearnCode               4949
LearnCodeOnline        16200
                       ...  
JobSatPoints_11        35992
SurveyLength            9255
SurveyEase              9199
ConvertedCompYearly    42002
JobSat                 36311
Length: 109, dtype: int64


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [4]:
# Write your code here
# Generate basic statistics for numerical columns only
numeric_stats = df.describe()

print(numeric_stats)


         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  65437.000000   3.374000e+04  29658.000000    29324.000000   
mean   32719.000000  2.963841e+145     11.466957       18.581094   
std    18890.179119  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16360.000000   6.000000e+04      4.000000        0.000000   
50%    32719.000000   1.100000e+05      9.000000       10.000000   
75%    49078.000000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.000000         0.00000   
25%          0.000000 

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [5]:
# Write your code here
import pandas as pd

# Assuming df is already loaded

# 1. Display unique values and their counts in 'Country' column (including NaNs)
print("Unique values and counts in 'Country':")
country_counts = df['Country'].value_counts(dropna=False)
print(country_counts)

# 2. Identify missing or empty string entries in 'Country'
missing_or_empty = df[df['Country'].isnull() | (df['Country'].str.strip() == '')]['Country']
print(f"\nNumber of missing or empty entries in 'Country': {missing_or_empty.shape[0]}")

# 3. Identify unusually short entries (length < 3 characters)
short_entries = df[df['Country'].notnull() & (df['Country'].str.len() < 3)]['Country'].unique()
print(f"\nShort country entries (length < 3): {short_entries}")

# 4. (Optional) Show all unique country names for manual inspection
unique_countries = df['Country'].dropna().unique()
print(f"\nTotal unique non-null country entries: {len(unique_countries)}")
print(unique_countries)


Unique values and counts in 'Country':
Country
United States of America                                11095
NaN                                                      6507
Germany                                                  4947
India                                                    4231
United Kingdom of Great Britain and Northern Ireland     3224
                                                        ...  
Micronesia, Federated States of...                          1
Nauru                                                       1
Chad                                                        1
Djibouti                                                    1
Solomon Islands                                             1
Name: count, Length: 186, dtype: int64

Number of missing or empty entries in 'Country': 6507

Short country entries (length < 3): []

Total unique non-null country entries: 185
['United States of America'
 'United Kingdom of Great Britain and Northern Ireland' 'Canada' 

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [6]:
## Write your code here
import pandas as pd

# Assuming df is already loaded

# 1. Identify inconsistent or irrelevant entries in 'Country'
print("Unique values and counts in 'Country':")
country_counts = df['Country'].value_counts(dropna=False)
print(country_counts)

missing_or_empty = df[df['Country'].isnull() | (df['Country'].str.strip() == '')]['Country']
print(f"\nNumber of missing or empty entries in 'Country': {missing_or_empty.shape[0]}")

short_entries = df[df['Country'].notnull() & (df['Country'].str.len() < 3)]['Country'].unique()
print(f"\nShort country entries (length < 3): {short_entries}")

unique_countries = df['Country'].dropna().unique()
print(f"\nTotal unique non-null country entries: {len(unique_countries)}")
print(unique_countries)

# 2. Standardize entries in 'Country' and 'EdLevel' columns by mapping inconsistent values

# Example mapping dictionary for Country
country_mapping = {
    'United States of America': 'USA',
    'United States': 'USA',
    'U.S.': 'USA',
    'USA': 'USA',
    'UK': 'United Kingdom',
    'U.K.': 'United Kingdom',
    'England': 'United Kingdom',
    'Great Britain': 'United Kingdom',
    # Add more mappings as needed
}

# Example mapping dictionary for EdLevel
edlevel_mapping = {
    'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 'Bachelor’s degree',
    'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 'Master’s degree',
    'Primary/elementary school': 'Primary/Elementary',
    'Secondary school (e.g. American high school, GCSEs, etc.)': 'Secondary school',
    'Some college/university study without earning a degree': 'Some college',
    # Add more mappings as needed
}

# Apply the mappings to create standardized columns
df['Country_Standardized'] = df['Country'].replace(country_mapping)
df['EdLevel_Standardized'] = df['EdLevel'].replace(edlevel_mapping)

# Check the results
print("\nSample standardized 'Country' values:")
print(df[['Country', 'Country_Standardized']].drop_duplicates().head(10))

print("\nSample standardized 'EdLevel' values:")
print(df[['EdLevel', 'EdLevel_Standardized']].drop_duplicates().head(10))


Unique values and counts in 'Country':
Country
United States of America                                11095
NaN                                                      6507
Germany                                                  4947
India                                                    4231
United Kingdom of Great Britain and Northern Ireland     3224
                                                        ...  
Micronesia, Federated States of...                          1
Nauru                                                       1
Chad                                                        1
Djibouti                                                    1
Solomon Islands                                             1
Name: count, Length: 186, dtype: int64

Number of missing or empty entries in 'Country': 6507

Short country entries (length < 3): []

Total unique non-null country entries: 185
['United States of America'
 'United Kingdom of Great Britain and Northern Ireland' 'Canada' 

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [7]:
## Write your code here
# One-hot encode the 'Employment' column
employment_dummies = pd.get_dummies(df['Employment'], prefix='Employment')

# Concatenate the new one-hot encoded columns to the original dataframe
df = pd.concat([df, employment_dummies], axis=1)

# Optional: view the first few rows of the new columns
print(df[employment_dummies.columns].head())


   Employment_Employed, full-time  \
0                            True   
1                            True   
2                            True   
3                           False   
4                           False   

   Employment_Employed, full-time;Employed, part-time  \
0                                              False    
1                                              False    
2                                              False    
3                                              False    
4                                              False    

   Employment_Employed, full-time;Independent contractor, freelancer, or self-employed  \
0                                              False                                     
1                                              False                                     
2                                              False                                     
3                                              False                      

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [8]:
## Write your code here
# Calculate missing values per column
missing_counts = df.isnull().sum()

# Sort columns by descending number of missing values
missing_counts_sorted = missing_counts.sort_values(ascending=False)

# Display the top columns with the most missing values (e.g., top 10)
print("Columns with the highest number of missing values:")
print(missing_counts_sorted.head(10))


Columns with the highest number of missing values:
AINextMuch less integrated       64289
AINextLess integrated            63082
AINextNo change                  52939
AINextMuch more integrated       51999
EmbeddedAdmired                  48704
EmbeddedWantToWorkWith           47837
EmbeddedHaveWorkedWith           43223
ConvertedCompYearly              42002
AIToolNot interested in Using    41023
AINextMore integrated            41009
dtype: int64


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [10]:
## Write your code here
# Impute missing values in ConvertedCompYearly with mean
mean_value = df['ConvertedCompYearly'].mean()
df['ConvertedCompYearly_mean_imputed'] = df['ConvertedCompYearly'].fillna(mean_value)

# Or impute missing values with median
median_value = df['ConvertedCompYearly'].median()
df['ConvertedCompYearly_median_imputed'] = df['ConvertedCompYearly'].fillna(median_value)

# Check the result
print(df[['ConvertedCompYearly', 'ConvertedCompYearly_mean_imputed', 'ConvertedCompYearly_median_imputed']].head(10))


   ConvertedCompYearly  ConvertedCompYearly_mean_imputed  \
0                  NaN                      86155.287263   
1                  NaN                      86155.287263   
2                  NaN                      86155.287263   
3                  NaN                      86155.287263   
4                  NaN                      86155.287263   
5                  NaN                      86155.287263   
6                  NaN                      86155.287263   
7                  NaN                      86155.287263   
8                  NaN                      86155.287263   
9                  NaN                      86155.287263   

   ConvertedCompYearly_median_imputed  
0                             65000.0  
1                             65000.0  
2                             65000.0  
3                             65000.0  
4                             65000.0  
5                             65000.0  
6                             65000.0  
7                  

<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [11]:
## Write your code here
# Find the most frequent value (mode) in RemoteWork
mode_value = df['RemoteWork'].mode()[0]

# Impute missing values with the mode
df['RemoteWork_imputed'] = df['RemoteWork'].fillna(mode_value)

# Verify
print(df[['RemoteWork', 'RemoteWork_imputed']].head(10))


  RemoteWork                    RemoteWork_imputed
0     Remote                                Remote
1     Remote                                Remote
2     Remote                                Remote
3        NaN  Hybrid (some remote, some in-person)
4        NaN  Hybrid (some remote, some in-person)
5        NaN  Hybrid (some remote, some in-person)
6     Remote                                Remote
7        NaN  Hybrid (some remote, some in-person)
8  In-person                             In-person
9     Remote                                Remote


### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [ ]:
## Write your code here

<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [ ]:
## Write your code here

### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [ ]:
## Write your code here

### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
